In [1]:
# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

DEBUG:__main__:hello


In [2]:
# ipynb形式のライブラリノートを.py形式に変更したものをインポート
import lib
import lib.lab_lib
from lib.lab_lib import *

DEBUG:lib.lab_lib:hello


# 2022年5月20日～

## 修正したモデルのMAPE表を作成

* ✅MAPE表の作成
* ✅修正済みモデルの追加前後で変化した関数の表
* 数値の右詰め

In [3]:
test_returnDFaboutDifferenceBetweenInput2DFs()

warnings.simplefilter("ignore")

benchmark_name: str = "cg"

classes: list[str] = ["A", "B", "C", "D", "E", "F"]
processes: list[int] = [2, 4, 8, 16, 32, 64, 128, 256]

dict_column_names_JP: dict[str, str] = {
    "functionName": "関数名",
    "modelLin": "線形",
    "modelIp": "反比例",
    "modelLog": "対数",
    "modelLinAndIp": "線形＆反比例",
    "modelLinAndLog": "線形＆対数",
    "modelIpAndLin": "反比例＆線形",
    "modelIpAndLog": "反比例＆対数",
    "modelLogAndLin": "対数＆線形",
    "modelLogAndIp": "対数＆反比例",
    "modelProcessDividedByProblemSize": "プロセス数を問題サイズで割る",
    "modelProblemSizeDividedByProcess": "問題サイズをプロセス数で割る",
    "modelLinearSumOf2elementCombination": "説明変数の要素の組み合わせの積の総和",
    # "modelLinearSumOfElementCombinations": "説明変数の要素の全組み合わせの積の線形和"
    # "modelBasicTree": "回帰木",
}

model_names_list_JP: list[str] = list(dict_column_names_JP.values())
model_names_list_JP.remove("関数名")

model_names_list: list[str] = list(dict_column_names_JP.keys())
model_names_list.remove("functionName")

date: str = "2022年5月25日"

# コア数と問題サイズの両方を振る

rawDF: pd.DataFrame = return_rawDF_with_init_param(
    benchmark_name=benchmark_name,
    classes=classes,
    processes=processes,
    csv_dir_path="./csv_files/",
)

exp_var: list[str] = rawDF.columns.tolist()
for element_be_removed in [
    "functionName",
    "functionCallNum",
    "intBenchmarkClass",
    "benchmarkName",
    "benchmarkClass",
]:
    exp_var.remove(element_be_removed)
res_var: list[str] = ["functionCallNum"]

print(f"exp_var = {exp_var}")

returned_MAPE_table: pd.DataFrame = return_MAPE_Table_DF_from_rawDF(
    rawDF=rawDF,
    exp_var_list=exp_var,
    res_var_list=res_var,
    model_name_list=model_names_list,
)
df: pd.DataFrame = returned_MAPE_table.rename(columns=dict_column_names_JP)
df = df.set_index("関数名")
df = df.sort_index(axis=0)
df = df.sort_index(axis=1)

dfBefore = addLowestMAPEsModelNameColumn(
    df, version=2, model_name_list=model_names_list_JP
)

benchmark_name: str = "cg"

classes: list[str] = ["A", "B", "C", "D", "E", "F"]
processes: list[int] = [2, 4, 8, 16, 32, 64, 128, 256]

dict_column_names_JP: dict[str, str] = {
    "functionName": "関数名",
    "modelLin": "線形",
    "modelIp": "反比例",
    "modelLog": "対数",
    "modelLinAndIp": "線形＆反比例",
    "modelLinAndLog": "線形＆対数",
    "modelIpAndLin": "反比例＆線形",
    "modelIpAndLog": "反比例＆対数",
    "modelLogAndLin": "対数＆線形",
    "modelLogAndIp": "対数＆反比例",
    "modelProcessDividedByProblemSize": "プロセス数を問題サイズで割る",
    "modelProblemSizeDividedByProcess": "問題サイズをプロセス数で割る",
    "modelLinearSumOf2elementCombination": "説明変数の要素の組み合わせの積の総和",
    "modelLinearSumOfElementCombinations": "説明変数の要素の全組み合わせの積の線形和"
    # "modelBasicTree": "回帰木",
}

model_names_list_JP: list[str] = list(dict_column_names_JP.values())
model_names_list_JP.remove("関数名")

model_names_list: list[str] = list(dict_column_names_JP.keys())
model_names_list.remove("functionName")

date: str = "2022年3月30日"

# コア数と問題サイズの両方を振る

rawDF: pd.DataFrame = return_rawDF_with_init_param(
    benchmark_name=benchmark_name,
    classes=classes,
    processes=processes,
    csv_dir_path="./csv_files/",
)

exp_var: list[str] = rawDF.columns.tolist()
for element_be_removed in [
    "functionName",
    "functionCallNum",
    "intBenchmarkClass",
    "benchmarkName",
    "benchmarkClass",
]:
    exp_var.remove(element_be_removed)
res_var: list[str] = ["functionCallNum"]

print(f"exp_var = {exp_var}")

returned_MAPE_table: pd.DataFrame = return_MAPE_Table_DF_from_rawDF(
    rawDF=rawDF,
    exp_var_list=exp_var,
    res_var_list=res_var,
    model_name_list=model_names_list,
)
df: pd.DataFrame = returned_MAPE_table.rename(columns=dict_column_names_JP)
df = df.set_index("関数名")
df = df.sort_index(axis=0)
df = df.sort_index(axis=1)

dfAfter = addLowestMAPEsModelNameColumn(
    df, version=2, model_name_list=model_names_list_JP
)


df = returnDFaboutDifferenceBetweenInput2DFs(dfBefore, dfAfter)
model_names_list: list[str] = df.columns.tolist()

result: pd.DataFrame = addLowestMAPEsModelNameColumn(df, model_names_list, version=2)

exp_var = ['process', 'na', 'nonzer', 'niter', 'shift']
exp_var = ['process', 'na', 'nonzer', 'niter', 'shift']


In [4]:
print(
    dfAfter.style.set_precision(2).to_latex(
        caption=f"ベンチマークプログラム{benchmark_name.upper()}の関数におけるすべてのモデルのMAPE",
        label=f"{date}allModelsMAPEin{benchmark_name.upper()}",
    )
)

dfAfter

\begin{table}
\caption{ベンチマークプログラムCGの関数におけるすべてのモデルのMAPE}
\label{2022年3月30日allModelsMAPEinCG}
\begin{tabular}{lrrrrrrrrrrrrrl}
 & プロセス数を問題サイズで割る & 反比例 & 反比例＆対数 & 反比例＆線形 & 問題サイズをプロセス数で割る & 対数 & 対数＆反比例 & 対数＆線形 & 線形 & 線形＆反比例 & 線形＆対数 & 説明変数の要素の全組み合わせの積の線形和 & 説明変数の要素の組み合わせの積の総和 & 最適モデル \\
関数名 &  &  &  &  &  &  &  &  &  &  &  &  &  &  \\
.TAU_application & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 線形 \\
ALLOC_SPACE & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 線形 \\
CG & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 線形 \\
CONJ_GRAD & 37.55 & 13.88 & 13.73 & 14.50 & 42.76 & 14.80 & 14.72 & 15.25 & 13.16 & 12.41 & 12.76 & 2.78 & 9.67 & 説明変数の要素の全組み合わせの積の線形和 \\
ICNVRT & 159.61 & 662.48 & 2504.27 & 894.57 & 119.98 & 2443.70 & 3811.49 & 815.79 & 502.12 & 6621.01 & 2283.06 & 35.33 & 84.75 & 説明変数の要素の全組み合わせの積の線形和 \\
INITIALIZE_MPI & 0.00 & 0.00 & 0.00 & 0.00 & 0.

,プロセス数を問題サイズで割る,反比例,反比例＆対数,反比例＆線形,問題サイズをプロセス数で割る,対数,対数＆反比例,対数＆線形,線形,線形＆反比例,線形＆対数,説明変数の要素の全組み合わせの積の線形和,説明変数の要素の組み合わせの積の総和,最適モデル
関数名,,,,,,,,,,,,,,
.TAU_application,7.733195e-09,0.000000,0.000000,0.000000,6.921169e-08,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,線形
ALLOC_SPACE,7.733195e-09,0.000000,0.000000,0.000000,6.921169e-08,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,線形
CG,7.733195e-09,0.000000,0.000000,0.000000,6.921169e-08,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,線形
CONJ_GRAD,3.755014e+01,13.876670,13.732936,14.498431,4.275719e+01,14.804389,14.717362,15.248570,13.161174,12.412559,12.755412,2.782334,9.666397,説明変数の要素の全組み合わせの積の線形和
ICNVRT,1.596142e+02,662.481358,2504.274677,894.574028,1.199846e+02,2443.699386,3811.490854,815.789372,502.123230,6621.011052,2283.060136,35.331868,84.751964,説明変数の要素の全組み合わせの積の線形和
INITIALIZE_MPI,7.733195e-09,0.000000,0.000000,0.000000,6.921169e-08,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,線形
MAKEA,7.733195e-09,0.000000,0.000000,0.000000,6.921169e-08,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,線形
MPI_Barrier(),7.733195e-09,0.000000,0.000000,0.000000,6.921169e-08,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,線形
MPI_Bcast(),7.733195e-09,0.000000,0.000000,0.000000,6.921169e-08,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,線形


In [5]:
print(
    result.style.set_precision(2).to_latex(
        caption=f"ベンチマークプログラム{benchmark_name.upper()}において新しく実装したモデルが最適モデルとなった関数のMAPE",
        label=f"{date}bestModelsMAPEin{benchmark_name.upper()}",
    )
)

result

\begin{table}
\caption{ベンチマークプログラムCGにおいて新しく実装したモデルが最適モデルとなった関数のMAPE}
\label{2022年3月30日bestModelsMAPEinCG}
\begin{tabular}{lrrl}
 & 説明変数の要素の全組み合わせの積の線形和 & 説明変数の要素の組み合わせの積の総和 & 最適モデル \\
関数名 &  &  &  \\
CONJ_GRAD & 2.78 & 9.67 & 説明変数の要素の全組み合わせの積の線形和 \\
ICNVRT & 35.33 & 84.75 & 説明変数の要素の全組み合わせの積の線形和 \\
MPI_Irecv() & 14.81 & 22.27 & 説明変数の要素の全組み合わせの積の線形和 \\
MPI_Send() & 14.81 & 22.27 & 説明変数の要素の全組み合わせの積の線形和 \\
MPI_Wait() & 14.81 & 22.27 & 説明変数の要素の全組み合わせの積の線形和 \\
SPRNVC & 31.22 & 62.48 & 説明変数の要素の全組み合わせの積の線形和 \\
VECSET & 31.22 & 62.48 & 説明変数の要素の全組み合わせの積の線形和 \\
\end{tabular}
\end{table}



,説明変数の要素の全組み合わせの積の線形和,説明変数の要素の組み合わせの積の総和,最適モデル
関数名,,,
CONJ_GRAD,2.782334,9.666397,説明変数の要素の全組み合わせの積の線形和
ICNVRT,35.331868,84.751964,説明変数の要素の全組み合わせの積の線形和
MPI_Irecv(),14.812929,22.272503,説明変数の要素の全組み合わせの積の線形和
MPI_Send(),14.812929,22.272503,説明変数の要素の全組み合わせの積の線形和
MPI_Wait(),14.812929,22.272503,説明変数の要素の全組み合わせの積の線形和
SPRNVC,31.218347,62.480987,説明変数の要素の全組み合わせの積の線形和
VECSET,31.218347,62.480987,説明変数の要素の全組み合わせの積の線形和


# 2022年4月17日～

次のような表を作成する

採用される割合 (MAPE の最大値 [%] ，MAPE の最小値 [%]) [%]

| ベンチマークプログラム名 | 線形モデル               | 対数モデル               | 反比例モデル              |
|--------------|---------------------|---------------------|---------------------|
| str          | float(float, float) | float(float, float) | float(float, float) |


目標となるのは一気にこのベンチマークプログラムを作成することだが、既存のライブラリ関数などを利用し、まずはベンチマークごとに作成可能にする。

メモ

## 実装予定

1. 行方向に最小値を検出
2. 最小値以外をNaNに変更
3. 列方向に最小値と最大値を検出

## 
